In [19]:
from sys import path

import numpy as np
import pandas as pd 
from scipy.stats import norm
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_theme()

%load_ext autoreload
%autoreload 2

# user-written 
import w8_estimation as est 
import w8_LinearModel as lm
import w8_probit as probit
import w8_logit as logit

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# Outcome label
y_lab = 'anyuseofforce_coded'

# Dataset columns
rawdat_columns = [
    'anyuseofforce_coded',
    
    # Subject (civilian) characteristics
    'sblack',
    'shisp',
    'swhite',
    'smale',
    'sage',
    
    # Officer characteristics
    'omajblack',
    'omajhisp',
    'omajwhite',
    
    # Encounter characteristics
    'daytime'
]

# Final X-matrix variable labels 
x_lab = [
    # Subject vars (white is reference)
    'sblack',
    'shisp',
    'smale',
    'sage',
    
    # Officer vars (white is reference)
    'omajblack',
    'omajhisp',

    # Encounter vars
    'daytime'
]

In [24]:
dat = pd.read_csv('ppcs_cc.csv')

N = dat.shape[0]

# reorder columns 
dat = dat[[y_lab] + x_lab].copy()

dat.head(5)

assert dat.notnull().all(axis=1).all(), 'Missings in the dataset, take them out!'

In [30]:
y = dat[y_lab].values
x = dat[x_lab].values
K = x.shape[1]

print("Shape x:", x.shape)
print("Rank x:", np.linalg.matrix_rank(x))

Shape x: (3799, 7)
Rank x: 7


In [26]:
# OLS estimates
ols_results = lm.estimate(y, x, robust_se=True)
ols_tab = lm.print_table((y_lab, x_lab), ols_results, title='LPM results')
ols_tab

LPM results
Dependent variable: anyuseofforce_coded

R2 = 0.005
sigma2 = nan


,b_hat,se,t
sblack,0.0054,0.0043,1.2496
shisp,0.0126,0.0063,1.9958
smale,0.0069,0.0022,3.0897
sage,-0.0000,0.0000,-0.3901
omajblack,-0.0051,0.0017,-3.0929
omajhisp,0.0038,0.0107,0.3515
daytime,-0.0009,0.0022,-0.4098


In [27]:
theta0 = probit.starting_values(y, x)
theta0

array([ 1.34510687e-02,  3.15588390e-02,  1.72939070e-02, -3.88903353e-05,
       -1.28120386e-02,  9.43479345e-03, -2.30021772e-03])

In [35]:
probit_results = est.estimate(probit.q, theta0, y, x, cov_type='Sandwich')

Optimization terminated successfully.
         Current function value: 0.092128
         Iterations: 1
         Function evaluations: 16
         Gradient evaluations: 2


LinAlgError: Singular matrix